In [6]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from quantfreedom.enums import *
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.indicators.indicators import qf_tv_calc_rsi_wilder
pd.options.plotting.backend = 'plotly'
mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
candles = mufex_main.get_candles(symbol="BTCUSDT", timeframe='5m', candles_to_dl=200)

It took 00 mins and 01 seconds to download 200 candles


In [4]:
np.around(qf_tv_calc_rsi_wilder(candles[:,4], period=14),4)

array([    nan,     nan,     nan,     nan,     nan,     nan,     nan,
           nan,     nan,     nan,     nan,     nan,     nan,     nan,
       72.8487, 63.9137, 69.459 , 72.4962, 73.9564, 70.4193, 63.0625,
       61.2427, 63.2212, 60.8075, 53.2597, 53.9743, 56.061 , 61.2779,
       51.7027, 52.867 , 57.2778, 52.8656, 54.177 , 59.0585, 58.3857,
       62.5859, 59.3447, 61.4264, 56.4926, 56.8894, 58.9334, 62.3938,
       56.048 , 55.4791, 58.9043, 68.6189, 80.9692, 82.0627, 68.7284,
       69.078 , 73.65  , 70.4058, 61.0598, 59.4978, 60.7968, 63.4809,
       60.436 , 59.8678, 61.8971, 62.3303, 61.28  , 57.8785, 57.299 ,
       53.9835, 56.6255, 57.8071, 55.8852, 58.4869, 62.8121, 61.1758,
       65.4788, 65.0756, 70.5039, 71.3247, 72.6612, 75.8418, 69.462 ,
       62.3292, 53.7269, 60.2392, 60.6692, 54.7575, 53.2493, 50.528 ,
       55.1523, 56.2865, 57.0659, 51.8762, 52.7186, 54.2053, 48.6528,
       47.5478, 47.1546, 43.1805, 44.9853, 42.0812, 45.0248, 40.423 ,
       44.5398, 48.1

In [7]:
pd.DataFrame(np.around(qf_tv_calc_rsi_wilder(candles[:,4], period=14),4)).plot()